In [90]:
import xarray as xr
import netCDF4
import numpy as np
import pandas as pd 
import pyeto as pt
import glob, os
import aasi as ai

In [91]:
# User-supplied
mapfile = '/glade/p/work/manab/ff/forcinggeneration/mappingislandpark.nc'
forcdir = '/glade/p/work/manab/ff/islandpark/rawinput'
concatdir = '/glade/p/work/manab/ff/islandpark/rawinputconcat/'
obsfile = '/glade/p/work/manab/ff/forcinggeneration/pilot_basin_data/isli_bor_flow_data.txt'
outdir = '/glade/p/work/manab/ff/islandpark/input/'
obsvar = 'dailydisc'

def ensConcat(dir, list, string):
    '''
    Searches for a string, concatenates all files for that ensemble,
    and writes out netCDF file
    '''
    enslist = []
    for count, value in enumerate(list):
        if value.endswith(string):
            enslist.append(value)
    ncconcat_time = xr.open_mfdataset(enslist, concat_dim='time')
    ncconcat_time.sortby('time')
    ncconcat_time.to_netcdf(dir+string)
        
        
def calcWeightedAvg(ncdat, varname):
    '''
    Calculates the weighted average 
    of any <varname> in the mapping file
    '''
    wAvg = sum(ncdat[varname]*ncdat['weight'])
    return(wAvg)

def calcPET(lat, time, tmin, tmax, tmean):
    '''
    Calculates Potential Evapotranspiration using 
    Hargreaves equation (Hargreaves and Samani, 1985) 
    '''
    
    latrad = pt.deg2rad(lat)                             #Latitude to radians
    dayofyear = pd.Series(time).dt.day.values
    etrad = []
    pet = []
    
    # Calculate ET radiation
    for x in np.nditer(dayofyear):
        soldec = pt.sol_dec(x)                           #Solar declination
        sha = pt.sunset_hour_angle(latrad, soldec)       #Sunset hour aingle
        ird = pt.inv_rel_dist_earth_sun(x)               #Inverse relative distance Earth-Sun
        etrad.append(pt.et_rad(latrad, soldec, sha, ird))#Extraterrestrial radiation
    
    # Calculate PET using hargreaves
    for x in range(0, len(etrad)):
        pet.append(pt.hargreaves(tmin[x], tmax[x], tmean[x], etrad[x]))
    
    pet = np.array(pet)
    return(pet)

def impqobs(file, starttime, endtime):
    '''
    Import Flow observation for the period
    '''
    dat = pd.read_table(file, skiprows = 14, header = None, 
                    names = ['time', 'gaugeheight', 'resinflow', 'dailydisc', 'unregflow'])
    dat['time'] = pd.to_datetime(dat['time'])
    dat = dat.set_index(['time'])   #Set time as index
    dat = dat.loc[starttime:endtime]
    return(dat)
    
def ncextract(ncfile):
    '''
    Open each forcing netCDF file
    '''
    forcdat = xr.open_dataset(ncfile)
    
    # Extracts the variables values from forcing file
    prcp = forcdat['prcp'].values
    tmax = forcdat['tmax'].values
    tmin = forcdat['tmin'].values
    time = forcdat['time'].values
    tmean = (tmax+tmin)/2
    return(forcdat, prcp, time, tmin, tmax, tmean)

In [92]:
if __name__ == '__main__':
    
    # Mapping file extraction
    mapdat = xr.open_dataset(mapfile)
    lat = calcWeightedAvg(mapdat, 'latitude')
    lon = calcWeightedAvg(mapdat, 'longitude')
    
    # Processing forcing files
    forcfiles = glob.glob(forcdir + '/*.nc') #List of all files
    forcfiles.sort()
    
    srchlist = glob.glob(forcdir + '/*ens_forc.PNW.0625.2010*.nc')   #Search string list 
    srchlist.sort()

    # Concatenates ensembles by time and exports to concatdir
    for count, value in enumerate(srchlist):
        srchstring = srchlist[count].split('.')[-2]+ '.' + srchlist[count].split('.')[-1] #Search string for each ensemble,002.nc
        ensConcat(concatdir, forcfiles, srchstring)
        print("Creating concatenated FUSE forcing file " , count+1, " of ", len(srchlist))
        
    # Processing forcing files
    concatforcfiles = glob.glob(concatdir + '/*.nc') #List of all files
    concatforcfiles.sort()
    
    # Processes all concatenated files and puts them in the final input directory
    for count, value in enumerate(concatforcfiles):
        [forcdat, prcp, time, tmin, tmax, tmean] = ncextract(value)
        
        pet = calcPET(lat, time, tmin, tmax, tmean)
        prcp = np.reshape(prcp,(prcp.shape[0],1,1))
        pet = np.reshape(pet,(pet.shape[0],1,1))
        tmean = np.reshape(tmax,(tmean.shape[0],1,1))

        #Extract observation for that period
        maxtime = forcdat['time'].max()
        mintime = forcdat['time'].min()
        obs = impqobs(obsfile, mintime, maxtime)
        obs = obs[obsvar].values
        obs = np.reshape(obs,(obs.shape[0],1,1))
        
        # Create new netCDF file
        f = xr.Dataset({'pr': (['time','latitude', 'longitude'],  prcp),
                        'pet': (['time','latitude', 'longitude'],  pet),
                        'temp': (['time','latitude', 'longitude'],  tmean),
                        'q_obs': (['time','latitude', 'longitude'],  obs)
                       },
                       coords={'time': time,
                               'latitude':[lat],
                               'longitude':[lon]}
                      )

        f.to_netcdf(outdir+os.path.basename(value))  
        print("Creating final FUSE forcing file " , count+1, " of ", len(concatforcfiles))

Creating concatenated FUSE forcing file  1  of  100
Creating concatenated FUSE forcing file  2  of  100
Creating concatenated FUSE forcing file  3  of  100
Creating concatenated FUSE forcing file  4  of  100
Creating concatenated FUSE forcing file  5  of  100
Creating concatenated FUSE forcing file  6  of  100
Creating concatenated FUSE forcing file  7  of  100
Creating concatenated FUSE forcing file  8  of  100
Creating concatenated FUSE forcing file  9  of  100
Creating concatenated FUSE forcing file  10  of  100
Creating concatenated FUSE forcing file  11  of  100
Creating concatenated FUSE forcing file  12  of  100
Creating concatenated FUSE forcing file  13  of  100
Creating concatenated FUSE forcing file  14  of  100
Creating concatenated FUSE forcing file  15  of  100
Creating concatenated FUSE forcing file  16  of  100
Creating concatenated FUSE forcing file  17  of  100
Creating concatenated FUSE forcing file  18  of  100
Creating concatenated FUSE forcing file  19  of  100
Cr

Creating final FUSE forcing file  65  of  100
Creating final FUSE forcing file  66  of  100
Creating final FUSE forcing file  67  of  100
Creating final FUSE forcing file  68  of  100
Creating final FUSE forcing file  69  of  100
Creating final FUSE forcing file  70  of  100
Creating final FUSE forcing file  71  of  100
Creating final FUSE forcing file  72  of  100
Creating final FUSE forcing file  73  of  100
Creating final FUSE forcing file  74  of  100
Creating final FUSE forcing file  75  of  100
Creating final FUSE forcing file  76  of  100
Creating final FUSE forcing file  77  of  100
Creating final FUSE forcing file  78  of  100
Creating final FUSE forcing file  79  of  100
Creating final FUSE forcing file  80  of  100
Creating final FUSE forcing file  81  of  100
Creating final FUSE forcing file  82  of  100
Creating final FUSE forcing file  83  of  100
Creating final FUSE forcing file  84  of  100
Creating final FUSE forcing file  85  of  100
Creating final FUSE forcing file  